In [ ]:
# General Utilities
import sys, os
import re
import pandas as pd
import numpy as np
np.random.seed(42)
#import en_core_sci_sm
import string
import unicodedata
import spacy
import nltk
nltk.download('stopwords');
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from flashtext import KeywordProcessor

stopwords = set(set(stopwords.words('english')) -set(["or"]))

# class
class text_preprocessing:
    def __init__(self):
        #self.nlp = en_core_sci_sm.load()
        self.nlp = spacy.load('en_core_sci_lg')
        
    def clean_sentence(self, text):
        """
        Purpose: Function to Clean the text
           1. Lowering the text
           2. Removing Punctuations
           3. Removing Numbers
           4. Lemmatization of text
           5. Removing Stopwords
           6. Removing Whitespaces
        Input: Raw string/text
        Output: Clean string/text
        """
        text = text.lower()
        text = unicodedata.normalize("NFKD", text)
        text = self.clean_text(text)
        text = self.remove_punct(text)
#         text = self.clean_numbers(text)
        text = self.lemma_text(text)
#         print(text)
        text = self.remove_stopwords(text, is_lower_case=True)
#         print(text)
        text = " ".join(text.split())
        text = text.replace("'", "")
        text = text.strip()
        return(text) 
    
    def clean_text(self, text):
        """Removing Punctuations from the text"""
        text = str(text)
        for punct in "/-'":
            text = text.replace(punct, ' ')
        for punct in '&':
            text = text.replace(punct, ' ')
        for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
            text = text.replace(punct, ' ')
        return(text)
    
    def clean_numbers(self, text):
        """Removing Numbers (0-9) from text"""
        if bool(re.search(r'\d', text)):
            text = re.sub('[0-9]', ' ', text)
        return(text)

    def remove_punct(self, text):
        clean_text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
        while '  ' in clean_text: clean_text = clean_text.replace('  ', ' ')
        return(clean_text)
    
    def lemma_text(self, text):
        """Converting the word into the root word (Lemmatization) using Scispacy Module"""
        s = [token.lemma_ for token in self.nlp(text) if token.lemma_ != '-PRON-']
        output = ' '.join(s)
        return(output)
    
    def remove_stopwords(self, text, is_lower_case=True):
        """Removing stopwords after tokenizing the text using ToktokTokenizer"""
        stopword_list = list(set(list(stopwords) + ['may','also','across','among','beside','however','yet','within']))
        tokenizer = ToktokTokenizer()
        tokens = tokenizer.tokenize(text)
        tokens = [token.strip() for token in tokens]
        if is_lower_case:
            filtered_tokens = [token for token in tokens if token not in stopword_list]
        else:
            filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
        filtered_text = " ".join(filtered_tokens)
        return(filtered_text)
    
    
class keyword_extract:
    def __init__(self, keyword_list):
        self.text_clean_device = text_preprocessing()
        self.original_keyword_list = keyword_list
        self.cleaned_keyword_list = [self.text_clean_device.clean_sentence(keyword) for keyword in self.original_keyword_list]
        self._keyword_map_dic = dict(zip(self.cleaned_keyword_list, self.original_keyword_list))
        self.keyword_processor = KeywordProcessor(case_sensitive=False)
        self.keyword_processor.add_keywords_from_list(self.cleaned_keyword_list)
        
    def _exact_match(self, text):
        reg_joinedpunct = re.compile(r'\d+/[a-zA-Z]+', re.S)
        sentence = text
        sentence = sentence.replace("â\x89¤", "less than equal to")
        sentence = sentence.replace("â\x89¥", "greater than equal to")
        sentence = re.sub("\n|\r", "", re.sub(" +", " ", sentence.strip()))
        sentence = re.sub('([.,;:!?{}()])', r' \1 ', sentence)
        sentence = re.sub('\s{2,}', ' ', sentence)
        sentence = reg_joinedpunct.sub(lambda m: m.group().replace("/", " ", 1), sentence)
        sentence = re.sub(r"(\d+) , *?(\d+)", r"\1,\2", sentence)
        sentence = re.sub(r"(\d+) \. *?(\d+)", r"\1.\2", sentence)
        sentence = re.sub("-", " ", sentence)
        keywords_found_list = sorted(list(set([keyword[0] for keyword in self.keyword_processor.extract_keywords(sentence, span_info=True)])))
        return(keywords_found_list)
           
    def extract_from(self, raw_text):
        clean_text = self.text_clean_device.clean_sentence(raw_text)
        keywords_found_list = self._exact_match(clean_text)
        original_keywords_map_list = [self._keyword_map_dic[keyword] for keyword in keywords_found_list]
        return(original_keywords_map_list)
 

class basic_keyword_extract:
    """Basic keyword extractor class using the keyword processor by Flashtext"""
    def __init__(self, keyword_list, case_sensitive=False):
        self.keyword_list = keyword_list
        self.case_sensitive = case_sensitive
        self.keyword_processor = self._initialize_basic_keyword_processor()
  
    def _initialize_basic_keyword_processor(self):
        kp = KeywordProcessor(case_sensitive=self.case_sensitive)
        kp.add_keywords_from_list(self.keyword_list)
        return kp
  
    def extract_from(self, raw_text):
        if self.case_sensitive: 
            text = raw_text
        else: 
            text = raw_text.lower()
        clean_text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
        while '  ' in clean_text: clean_text = clean_text.replace('  ', ' ')
        extracted_keywords_ls1 = self.keyword_processor.extract_keywords(text)
        extracted_keywords_ls2 = self.keyword_processor.extract_keywords(clean_text)
        return(extracted_keywords_ls1+extracted_keywords_ls2)


def create_map_from_df(df, key_column, value_column):
    df_temp = df[[key_column, value_column]].dropna()
    df_temp = df_temp.apply(lambda x: x.astype(str).str.strip())
    df_temp = df_temp.drop_duplicates().reset_index(drop=True)
    map_dict = dict(zip(df_temp[key_column].str.lower(), df_temp[value_column]))
    return(map_dict)


def extract_key_column(df, desc_col_name, key_column_keyword_df, advance_keyword_extractor=False, default_key_column_raw=None, default_key_column_standard=None, default_key_column_group=None):
    df_copy = df.copy()
    # Column names
    _col__key_column_measure_type = 'key_column_measure_type'
    _col__standardized_key_column_name = 'standardized_key_column_name'
    _col__raw_keywords = 'raw_keywords'
    temp_key_column_keyword_df = key_column_keyword_df.copy()

    # Map dict between raw keywords and standard form + standard and group
    raw_to_standard_map_dict = create_map_from_df(temp_key_column_keyword_df, _col__raw_keywords, _col__standardized_key_column_name)

    if default_key_column_raw and default_key_column_standard and default_key_column_group and (default_key_column_raw.lower() not in raw_to_standard_map_dict.keys()):
        raw_to_standard_map_dict[default_key_column_raw] = default_key_column_standard
#         raw_to_standard_map_dict[default_key_column_raw.lower()] = default_key_column_standard

    raw_to_group_map_dict = create_map_from_df(temp_key_column_keyword_df, _col__raw_keywords, _col__key_column_measure_type)
  
    if default_key_column_raw and default_key_column_standard and default_key_column_group and (default_key_column_raw.lower() not in raw_to_group_map_dict.keys()):
        raw_to_group_map_dict[default_key_column_raw] = default_key_column_group
#         raw_to_group_map_dict[default_key_column_raw.lower()] = default_key_column_group

    # List of unique keywords
    key_column_keyword_ls = list(set(str(i).lower().strip() for i in temp_key_column_keyword_df[_col__raw_keywords].unique().tolist()))

    # Intialize keyword processor
    if advance_keyword_extractor:
        end_extract_kp = keyword_extract(key_column_keyword_ls)
    else:
        end_extract_kp = basic_keyword_extract(key_column_keyword_ls, case_sensitive=False)
    
    # Extraction from data
    df_copy['extracted_key_column'] = df_copy[desc_col_name].apply(lambda x: np.nan if(pd.isna(x)) else '|'.join(set(e.lower().strip() for e in end_extract_kp.extract_from(x) if(e.strip()!=''))))

    df_copy['extracted_key_column'] = df_copy['extracted_key_column'].replace({'  ':np.nan,' ':np.nan,'':np.nan,'|':np.nan})
    if default_key_column_raw and default_key_column_standard and default_key_column_group:
        df_copy['extracted_key_column'] = df_copy['extracted_key_column'].fillna(default_key_column_raw)
#         df_copy['extracted_key_column'] = df_copy['extracted_key_column'].fillna(default_key_column_raw.lower())
    
    
    #extra parts added
    def _get_standard_endp(text):
        endp_list = str(text).strip().split("|")
        std_list = []
        group_list = []
        for endp in endp_list:
            endp = endp.strip()
            if endp in raw_to_standard_map_dict:
                st_end = raw_to_standard_map_dict[endp]
            else:
                st_end = 'Not Available'
                
                
            if endp in raw_to_group_map_dict:
                grp = raw_to_group_map_dict[endp]
            else:
                grp = 'Not Available'
                
            std_list.append(st_end)
            group_list.append(grp)
            
        temp_dict = {'st_endp':"|".join(std_list), 'group':'|'.join(group_list)}
        return temp_dict
        
            
    df_pivot_down = df_copy.copy()
    df_pivot_down.reset_index(drop=True, inplace=True)
#     df_pivot_down = df_pivot_down.explode('extracted_key_column')
    df_pivot_down = df_pivot_down.assign(extracted_key_column=df_pivot_down['extracted_key_column'].str.split('|')).explode('extracted_key_column')
    df_pivot_down['standardized_key_column'] = df_pivot_down['extracted_key_column'].apply(lambda x: _get_standard_endp(x)['st_endp'])
    df_pivot_down['key_column_group'] = df_pivot_down['extracted_key_column'].apply(lambda x: _get_standard_endp(x)['group'])
    
    df_copy['standardized_key_column'] = df_copy['extracted_key_column'].apply(lambda x: _get_standard_endp(x)['st_endp'])
    df_copy['key_column_group'] = df_copy['extracted_key_column'].apply(lambda x: _get_standard_endp(x)['group'])
    
    
    
    return df_copy, df_pivot_down
    
    
    
    
#     # Roll Down at extracted scale
#     df_roll_down = df_copy.assign(extracted_key_column=df_copy['extracted_key_column'].str.split('|')).explode('extracted_key_column')
#     df_roll_down = df_roll_down.drop(columns=[desc_col_name])
#     df_roll_down = df_roll_down.replace('  ', ' ').replace(' ', '').replace('', np.nan).fillna(np.nan)
#     df_roll_down = df_roll_down.dropna(subset=['extracted_key_column'])
#     df_roll_down = df_roll_down.drop_duplicates().reset_index(drop=True)
#     df_roll_down['standardized_key_column'] = df_roll_down['extracted_key_column'].apply(lambda x: np.nan if(pd.isna(x)) else raw_to_standard_map_dict[x] if(x in raw_to_standard_map_dict.keys()) else np.nan)
#     df_roll_down['key_column_group'] = df_roll_down['extracted_key_column'].apply(lambda x: np.nan if(pd.isna(x)) else raw_to_group_map_dict[x] if(x in raw_to_group_map_dict.keys()) else np.nan)
#     return df_copy
#     return(df_roll_down)


# if __name__ == '__main__':
#     key_column_description_df = 
#     standardized_key_column_df = 

#     key_column_rolldown_df = extract_key_column(df=key_column_description_df,
#                                             desc_col_name='key_column_description_text',
#                                             key_column_keyword_df=standardized_key_column_df,
#                                             advance_keyword_extractor=True)

In [ ]:
df_mapping = pd.read_excel("RAW_key_column_STANDARDIZED_key_column_MAPPING_MASTER_DATABASE.xlsx", skiprows=2)

In [ ]:
df_mapping = df_mapping[['key_column', 'Standardized key_column','key_column Group']]

In [ ]:
df_mapping.shape

In [ ]:
df_temp = df_mapping[['Standardized key_column (SME Driven)', 'key_column Group']]

df_temp['key_column'] = df_temp['Standardized key_column']

In [ ]:
df_mapping = df_mapping.append(df_temp, sort=True, ignore_index=True)

In [ ]:
df_mapping.shape

In [ ]:
df_key_column = pd.read_excel("key_column for standardization.xlsx")

In [ ]:
df_key_column.shape

In [ ]:
df_mapping.columns

In [ ]:
standardized_key_column_df = df_mapping.copy()
standardized_key_column_df.rename({'key_column':'raw_keywords', 'Standardized key_column':'standardized_key_column_name', 'key_column Group':'key_column_measure_type'}, axis=1, inplace=True)

In [ ]:
not_available = "Not Available"

In [ ]:
df_endp_std, df_pivot_down =  extract_key_column(df=df_key_column,
                                desc_col_name='key_column Name',
                                key_column_keyword_df=standardized_key_column_df,
                                advance_keyword_extractor=True,
                                default_key_column_group=not_available,
                                default_key_column_standard=not_available,
                                default_key_column_raw=not_available)

In [ ]:
df_endp_std.shape

In [ ]:
df_pivot_down.shape

In [ ]:
df_endp_std.to_excel("std_key_column_output.xlsx", index=False)
df_pivot_down.to_excel("std_key_column_output_pivot_down.xlsx", index=False)